In [ ]:
from mintalib import core
from mintalib import functions as fx


class PricesMethods:
    _prices = None

    def dispatch_prices(self, func, *args, **kwds):
        result = func(self._prices, *args, **kwds)
        return core.wrap_result(result, self._prices)

    def atr(self, period: int = 14):
        kwds = dict(period=period)
        return self.dispatch_prices(core.calc_atr, **kwds)



class SeriesMethods:
    _series = None

    def dispatch_series(self, func, *args, **kwds):
        result = func(self._series, *args, **kwds)
        return core.wrap_result(result, self._series)

    def ema(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_ema, **kwds)

    def sma(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_sma, **kwds)


def register_pandas(name="ta"):
    import pandas as pd

    @pd.api.extensions.register_dataframe_accessor(name)
    class PricesAccessor(PricesMethods):
        def __init__(self, prices):
            self._validate(prices)
            self._prices = prices

        @staticmethod
        def _validate(prices):
            if "close" not in prices.columns:
                raise AttributeError("Prices must have a 'close' column") 

        @property
        def _series(self):
            return self._prices


    @pd.api.extensions.register_series_accessor(name)
    class SeriesAccessor(SeriesMethods):
        def __init__(self, series):
            self._series = series


register_pandas()


/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_19838/2253925272.py:38: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.PricesAccessor'> under name 'ta' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PricesAccessor(PricesMethods):
/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_19838/2253925272.py:54: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.SeriesAccessor'> under name 'ta' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  class SeriesAccessor(SeriesMethods):


In [13]:
import yfinance as yf
from mintalib.indicators import SMA

# fetch prices (eg with yfinance)
prices = yf.Ticker('AAPL').history('5y')

# convert column and index names to lower case
prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)

# compute and append indicators to prices
result = prices.assign(
    sma50 = SMA(50),
)
result

,open,high,low,close,volume,dividends,stock splits,sma50
date,,,,,,,,
2020-01-13 00:00:00-05:00,75.532394,76.848465,75.413627,76.821800,121532000,0.0,0.0,NaN
2020-01-14 00:00:00-05:00,76.758774,76.969635,75.660835,75.784439,161954400,0.0,0.0,NaN
2020-01-15 00:00:00-05:00,75.583290,76.467941,75.025833,75.459679,121923600,0.0,0.0,NaN
2020-01-16 00:00:00-05:00,76.005012,76.516418,75.641456,76.404922,108829200,0.0,0.0,NaN
2020-01-17 00:00:00-05:00,76.654541,77.253197,76.346733,77.250778,137816400,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2025-01-03 00:00:00-05:00,243.360001,244.179993,241.889999,243.360001,40244100,0.0,0.0,237.840298
2025-01-06 00:00:00-05:00,244.309998,247.330002,243.199997,245.000000,45045600,0.0,0.0,238.130170
2025-01-07 00:00:00-05:00,242.979996,245.550003,241.350006,242.210007,40856000,0.0,0.0,238.368038


In [14]:
prices.ta.atr()

date
2020-01-13 00:00:00-05:00         NaN
2020-01-14 00:00:00-05:00         NaN
2020-01-15 00:00:00-05:00         NaN
2020-01-16 00:00:00-05:00         NaN
2020-01-17 00:00:00-05:00         NaN
                               ...   
2025-01-03 00:00:00-05:00    4.280563
2025-01-06 00:00:00-05:00    4.269809
2025-01-07 00:00:00-05:00    4.264822
2025-01-08 00:00:00-05:00    4.221621
2025-01-10 00:00:00-05:00    4.612934
Length: 1257, dtype: float64

In [15]:
prices.close.ta.sma()

AttributeError: 'SeriesAccessor' object has no attribute '_prices'

In [11]:
prices.ta.sma(50)

AttributeError: 'PricesAccessor' object has no attribute 'sma'

In [29]:
prices.ta.ema()

In [30]:
import inspect
def binding_wrapper(func):
    signature = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = signature.bind(*args, **kwargs)
        bound.apply_defaults()
        return bound.arguments
    return wrapper

@binding_wrapper
def myfunc(a=1, b=2):
    return dict(a=a, b=b)

myfunc()

{'a': 1, 'b': 2}